In [12]:
import yahoo_fin.stock_info as si
import pandas as pd
import numpy as np
import concurrent.futures
import time
startTime = time.time()

In [13]:
def pull_fin_info(function_call, stock_list):
    futures_df = {}
    with concurrent.futures.ProcessPoolExecutor() as executor:
        for ticker in stock_list:
                futures_df[ticker] = executor.submit(function_call,ticker)

    results_df = {}
    for ticker in stock_list:
        try:
            results_df[ticker] = futures_df[ticker].result(60) # 60 sec timeout
        except:
            print('pull error on ' + ticker)
            break
    
    # print(results_df)

    recent_sheets = pd.DataFrame(np.empty((0, 1)))
    recent_sheets = {ticker : sheet.iloc[:,:1] for ticker,sheet in results_df.items()}
    for ticker in recent_sheets.keys():
        # if recent_sheets[ticker].columns() > 0:
        try:
            recent_sheets[ticker].columns = ['Recent']
        except ValueError:
            print('column re-name error on ' + ticker)

    recent_sheets = pd.concat(recent_sheets)
    recent_sheets = recent_sheets.reset_index()
    recent_sheets.columns = ['Ticker', 'Breakdown', 'Recent']
    return recent_sheets

In [14]:
stock_list = ['AMZN','KO','TSLA','GME','AAPL','GOOG','SPOT']
stock_list.extend(['ELV'])

stock_list.extend(si.tickers_dow())
# stock_list.extend(si.tickers_sp500())
print('stock list pull done seconds - ' + str(round(time.time() - startTime,2)))

stock_list = list(dict.fromkeys(stock_list))
# print(stock_list)

stock list pull done seconds - 0.46


In [15]:
balance_sheets = pull_fin_info(si.get_balance_sheet, stock_list)
print('balance sheets done seconds - ' + str(round(time.time() - startTime,2)))

balance sheets done seconds - 10.0


In [16]:
income_statements = pull_fin_info(si.get_income_statement, stock_list)
print('income statements done seconds - ' + str(round(time.time() - startTime,2)))


income statements done seconds - 29.98


In [17]:
cash_flow_statements = pull_fin_info(si.get_cash_flow, stock_list)
print('cash flow statements done seconds - ' + str(round(time.time() - startTime,2)))


cash flow statements done seconds - 50.52


In [18]:
frames = [balance_sheets, income_statements, cash_flow_statements]
combined_df = pd.concat(frames)
combined_df = combined_df.reset_index()

In [19]:
pivot_df = combined_df.pivot_table(index='Ticker', columns='Breakdown', values='Recent', aggfunc='sum')

In [20]:
pivot_df.query("Ticker == 'AMZN'")

Breakdown,accountsPayable,capitalExpenditures,capitalSurplus,cash,changeInCash,changeToAccountReceivables,changeToInventory,changeToLiabilities,changeToNetincome,changeToOperatingActivities,...,totalCashFromOperatingActivities,totalCashflowsFromInvestingActivities,totalCurrentAssets,totalCurrentLiabilities,totalLiab,totalOperatingExpenses,totalOtherIncomeExpenseNet,totalRevenue,totalStockholderEquity,treasuryStock
Ticker,,,,,,,,,,,,,,,,,,,,,
AMZN,7.866400e+10,-6.105300e+10,5.553800e+10,3.622000e+10,-5.900000e+09,-1.816300e+10,-9.487000e+09,5.916000e+09,-1.722000e+09,2.123000e+09,...,4.632700e+10,-5.815400e+10,1.615800e+11,1.422660e+11,2.823040e+11,4.449430e+11,1.327600e+10,4.698220e+11,1.382450e+11,-3.213000e+09


In [21]:
pivot_df.query("ebit >= 0")

Breakdown,accountsPayable,capitalExpenditures,capitalSurplus,cash,changeInCash,changeToAccountReceivables,changeToInventory,changeToLiabilities,changeToNetincome,changeToOperatingActivities,...,totalCashFromOperatingActivities,totalCashflowsFromInvestingActivities,totalCurrentAssets,totalCurrentLiabilities,totalLiab,totalOperatingExpenses,totalOtherIncomeExpenseNet,totalRevenue,totalStockholderEquity,treasuryStock
Ticker,,,,,,,,,,,,,,,,,,,,,
AAPL,5.476300e+10,-1.108500e+10,NaN,3.494000e+10,-3.860000e+09,-1.012500e+10,-2.642000e+09,1.400200e+10,2.985000e+09,-6.146000e+09,...,1.040380e+11,-1.454500e+10,1.348360e+11,1.254810e+11,2.879120e+11,2.568680e+11,2.580000e+08,3.658170e+11,6.309000e+10,1.630000e+08
AMGN,1.366000e+09,-8.800000e+08,NaN,7.989000e+09,1.723000e+09,-4.290000e+08,-1.650000e+08,-6.900000e+07,1.318000e+09,1.190000e+08,...,9.261000e+09,7.330000e+08,1.938500e+10,1.218400e+10,5.446500e+10,1.664100e+10,-2.637000e+09,2.597900e+10,6.700000e+09,-7.960000e+08
AMZN,7.866400e+10,-6.105300e+10,5.553800e+10,3.622000e+10,-5.900000e+09,-1.816300e+10,-9.487000e+09,5.916000e+09,-1.722000e+09,2.123000e+09,...,4.632700e+10,-5.815400e+10,1.615800e+11,1.422660e+11,2.823040e+11,4.449430e+11,1.327600e+10,4.698220e+11,1.382450e+11,-3.213000e+09
AXP,9.495600e+10,-1.638000e+09,1.149500e+10,2.104000e+10,-1.093700e+10,NaN,NaN,5.389000e+09,-1.567000e+09,1.068000e+09,...,1.464500e+10,-1.052900e+10,1.640180e+11,1.058250e+11,1.663710e+11,3.363000e+10,5.200000e+08,4.379900e+10,2.217700e+10,-2.945000e+09
CAT,8.154000e+09,-2.472000e+09,NaN,8.428000e+09,-1.030000e+08,-1.259000e+09,-2.586000e+09,2.075000e+09,-9.110000e+08,1.123000e+09,...,7.198000e+09,-3.084000e+09,4.345500e+10,2.984700e+10,6.627700e+10,4.269700e+10,-3.900000e+07,5.097100e+10,1.648400e+10,-2.919600e+10
CRM,2.956000e+09,-7.170000e+08,5.091900e+10,5.464000e+09,-7.310000e+08,-1.824000e+09,NaN,3.136000e+09,1.568000e+09,-2.970000e+09,...,6.000000e+09,-1.453600e+10,2.285000e+10,2.178800e+10,3.707800e+10,2.589000e+10,9.300000e+08,2.649200e+10,5.813100e+10,-1.660000e+08
CSCO,2.513000e+09,-6.920000e+08,NaN,9.175000e+09,-1.870000e+09,-1.070000e+08,-2.440000e+08,1.507000e+09,1.148000e+09,1.377000e+09,...,1.545400e+10,-5.285000e+09,3.911200e+10,2.625700e+10,5.622200e+10,3.605300e+10,-5.030000e+08,4.981800e+10,4.127500e+10,-4.170000e+08
CVX,1.645400e+10,-8.056000e+09,1.728200e+10,5.640000e+09,5.800000e+07,-7.548000e+09,-5.300000e+08,5.475000e+09,-2.323000e+09,-2.800000e+08,...,2.918700e+10,-5.865000e+09,3.373800e+10,2.679100e+10,9.959500e+10,1.397460e+11,5.779000e+09,1.556060e+11,1.390670e+11,-4.559300e+10
DIS,1.567900e+10,-3.578000e+09,NaN,1.595900e+10,-1.951000e+09,-3.570000e+08,2.520000e+08,2.410000e+09,-3.733000e+09,1.710000e+08,...,5.567000e+09,-3.163000e+09,3.365700e+10,3.107700e+10,1.013850e+11,6.392600e+10,-9.310000e+08,6.741800e+10,8.855300e+10,-7.347000e+09


In [22]:
executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(round(executionTime,2)))

Execution time in seconds: 50.69
